<img src="https://raw.githubusercontent.com/rbizoi/PythonFormation/main/images/e-brasil.png" width="850">


https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

# Les imports et initialisations des variables

In [1]:
from datetime import datetime
import pandas as pd, numpy as np, os, warnings, seaborn as sns, pickle, re, unicodedata
from datetime import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font1 = fm.FontProperties(size=20)
font2 = fm.FontProperties(size=24)

%matplotlib inline

if int(str(sns.__version__).split('.')[1]) > 8 : 
    plt.style.use('seaborn-v0_8-darkgrid')
else:
    plt.style.use('seaborn-darkgrid')
    
sns.set(font_scale=2)
warnings.filterwarnings(action="ignore")

## Liste fichiers de données 

In [2]:
!dir ..\donnees\ebrasil

Le fichier spécifié est introuvable.


## Changement de répertoire

In [3]:
#os.chdir(r"../donnees")
os.chdir("C:/Users/etien/Documents/CCI/_HUB/cours/2024-2025/big_data/_git/donnees")

# DataFrame $reviews$

In [4]:
donnees = pd.read_csv(os.path.join('ebrasil', 'olist_order_reviews_dataset.csv'))
donnees.rename(columns={col:col.replace('review_','') for col in donnees.columns[2:]},inplace=True)
donnees.creation_date = pd.to_datetime(donnees.creation_date, format='%Y-%m-%d %H:%M:%S',errors='ignore')
donnees.answer_timestamp = pd.to_datetime(donnees.answer_timestamp, format='%Y-%m-%d %H:%M:%S',errors='ignore')
donnees = donnees.merge(pd.read_parquet('./ecommerce/orders.parquet')[['order_id', 'purchase_timestamp']],on='order_id')

In [5]:
donnees.head()

,review_id,order_id,score,comment_title,comment_message,creation_date,answer_timestamp,purchase_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18,2018-01-18 21:46:59,2018-01-11 15:30:49
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10,2018-03-11 03:05:13,2018-02-28 12:25:19
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17,2018-02-18 14:36:24,2018-02-03 09:56:22
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06,2017-04-09 17:41:13
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53,2018-02-10 10:59:03


In [6]:
donnees['creation'] = ((donnees.creation_date - donnees.purchase_timestamp).dt.days*24+(donnees.creation_date - donnees.purchase_timestamp).dt.seconds/60/60).astype('int32')
donnees['answer'] = ((donnees.answer_timestamp - donnees.creation_date).dt.days*24+(donnees.answer_timestamp - donnees.creation_date).dt.seconds/60/60).astype('int32')
donnees.score = donnees.score.astype('int16')
donnees['comment'] = donnees.comment_message.apply(lambda x : len(str(x))).astype('int16')
donnees.head()

,review_id,order_id,score,comment_title,comment_message,creation_date,answer_timestamp,purchase_timestamp,creation,answer,comment
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18,2018-01-18 21:46:59,2018-01-11 15:30:49,152,21,3
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10,2018-03-11 03:05:13,2018-02-28 12:25:19,227,27,3
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17,2018-02-18 14:36:24,2018-02-03 09:56:22,326,38,3
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06,2017-04-09 17:41:13,270,22,37
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53,2018-02-10 10:59:03,445,34,100


In [7]:
donnees.order_id.nunique(),donnees.shape[0]

(99441, 100000)

In [8]:
donnees.dtypes

review_id                     object
order_id                      object
score                          int16
comment_title                 object
comment_message               object
creation_date         datetime64[ns]
answer_timestamp      datetime64[ns]
purchase_timestamp    datetime64[ns]
creation                       int32
answer                         int32
comment                        int16
dtype: object

In [9]:
donnees.to_parquet('./ecommerce/reviews.parquet',compression='gzip', engine='pyarrow')

In [10]:
donnees.pivot_table(index='order_id',
                    columns='score',
                    values=['review_id','creation','answer','comment'], 
                    aggfunc={'review_id':'count','creation':'sum','answer':'sum','comment':'sum'},
                    fill_value=0)

answer               comment               \
score                                 1  2  3   4   5       1  2  3  4   5   
order_id                                                                     
00010242fe8c5a6d1ba2dd792cb16214      0  0  0   0  34       0  0  0  0  46   
00018f77f2f0320c557190d7a144bdd3      0  0  0  59   0       0  0  0  3   0   
000229ec398224ef6ca0657da4fc703e      0  0  0   0  16       0  0  0  0  90   
00024acbcdf0a6daa1e931b038114c75      0  0  0  16   0       0  0  0  3   0   
00042b26cf59d7ce69dfabb4e55b4fd9      0  0  0   0  34       0  0  0  0  39   
...                                 ... .. ..  ..  ..     ... .. .. ..  ..   
fffc94f6ce00a00581880bf54a75a037      0  0  0   0  84       0  0  0  0   3   
fffcd46ef2263f404302a634eb57f7eb      0  0  0   0  33       0  0  0  0   3   
fffce4705a9662cd70adb13d4a31832d      0  0  0   0  21       0  0  0  0   3   
fffe18544ffabc95dfada21779c9644f      0  0  0   0  36       0  0  0  0  47   
fffe41c64501cc87c801fd61db3f6244      0  0  0   0  69       0  0  0  0   3   

                                 creation                 review_id           \
score                                   1  2  3    4    5         1  2  3  4   
order_id                                                                       
00010242fe8c5a6d1ba2dd792cb16214        0  0  0    0  183         0  0  0  0   
00018f77f2f0320c557190d7a144bdd3        0  0  0  397    0         0  0  0  1   
000229ec398224ef6ca0657da4fc703e        0  0  0    0  201         0  0  0  0   
00024acbcdf0a6daa1e931b038114c75        0  0  0  157    0         0  0  0  1   
00042b26cf59d7ce69dfabb4e55b4fd9        0  0  0    0  610         0  0  0  0   
...                                   ... .. ..  ...  ...       ... .. .. ..   
fffc94f6ce00a00581880bf54a75a037        0  0  0    0  418         0  0  0  0   
fffcd46ef2263f404302a634eb57f7eb        0  0  0    0  229         0  0  0  0   
fffce4705a9662cd70adb13d4a31832d        0  0  0    0  126         0  0  0  0   
fffe18544ffabc95dfada21779c9644f        0  0  0    0   48         0  0  0  0   
fffe41c64501cc87c801fd61db3f6244        0  0  0    0  126         0  0  0  0   

                                     
score                             5  
order_id                             
00010242fe8c5a6d1ba2dd792cb16214  1  
00018f77f2f0320c557190d7a144bdd3  0  
000229ec398224ef6ca0657da4fc703e  1  
00024acbcdf0a6daa1e931b038114c75  0  
00042b26cf59d7ce69dfabb4e55b4fd9  1  
...                              ..  
fffc94f6ce00a00581880bf54a75a037  1  
fffcd46ef2263f404302a634eb57f7eb  1  
fffce4705a9662cd70adb13d4a31832d  1  
fffe18544ffabc95dfada21779c9644f  1  
fffe41c64501cc87c801fd61db3f6244  1  

[99441 rows x 20 columns]

In [11]:
donnees = donnees.pivot_table(index='order_id',
                    columns='score',
                    values=['review_id','creation','answer','comment'], 
                    aggfunc={'review_id':'count','creation':'sum','answer':'sum','comment':'sum'},
                    fill_value=0)
donnees.columns = [f"{col[0].replace('review_id','score')}_{col[1]}" for col in donnees.columns]
donnees.reset_index(inplace=True)
donnees['score'] = donnees.score_1+\
                   donnees.score_2+\
                   donnees.score_3+\
                   donnees.score_4+\
                   donnees.score_5

donnees['answer'] = donnees.answer_1+\
                   donnees.answer_2+\
                   donnees.answer_3+\
                   donnees.answer_4+\
                   donnees.answer_5

donnees['creation'] = donnees.creation_1+\
                      donnees.creation_2+\
                      donnees.creation_3+\
                      donnees.creation_4+\
                      donnees.creation_5

donnees['comment'] = donnees.comment_1+\
                   donnees.comment_2+\
                   donnees.comment_3+\
                   donnees.comment_4+\
                   donnees.comment_5
donnees.head()

,order_id,answer_1,answer_2,answer_3,answer_4,answer_5,comment_1,comment_2,comment_3,comment_4,...,creation_5,score_1,score_2,score_3,score_4,score_5,score,answer,creation,comment
0,00010242fe8c5a6d1ba2dd792cb16214,0,0,0,0,34,0,0,0,0,...,183,0,0,0,0,1,1,34,183,46
1,00018f77f2f0320c557190d7a144bdd3,0,0,0,59,0,0,0,0,3,...,0,0,0,0,1,0,1,59,397,3
2,000229ec398224ef6ca0657da4fc703e,0,0,0,0,16,0,0,0,0,...,201,0,0,0,0,1,1,16,201,90
3,00024acbcdf0a6daa1e931b038114c75,0,0,0,16,0,0,0,0,3,...,0,0,0,0,1,0,1,16,157,3
4,00042b26cf59d7ce69dfabb4e55b4fd9,0,0,0,0,34,0,0,0,0,...,610,0,0,0,0,1,1,34,610,39


In [12]:
donnees.to_parquet('./ecommerce/reviews_p.parquet',compression='gzip', engine='pyarrow')

In [13]:
!dir ecommerce

 Le volume dans le lecteur C s’appelle Windows-SSD
 Le numéro de série du volume est 84DC-AF7F

 Répertoire de C:\Users\etien\Documents\CCI\_HUB\cours\2024-2025\big_data\_git\donnees\ecommerce

20/11/2024  16:24    <DIR>          .
20/11/2024  15:02    <DIR>          ..
21/11/2024  09:22         4 807 544 items.parquet
21/11/2024  09:22         9 567 921 orders.parquet
21/11/2024  09:23        10 022 487 orders_payments.parquet
21/11/2024  09:23        13 281 965 orders_payments_items.parquet
21/11/2024  09:22         2 651 184 payments.parquet
21/11/2024  09:22         2 470 995 paymentsI.parquet
21/11/2024  09:23         7 549 835 reviews.parquet
21/11/2024  09:23         2 845 170 reviews_p.parquet
               8 fichier(s)       53 197 101 octets
               2 Rép(s)  709 470 416 896 octets libres
